In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [13]:
from crewai import Crew, Process # Process class let's you define different ways for agents to work together
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)


In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}


In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
"context": "The goal is to identify trends and predict market movements for AAPL. Please use all available data sources and analytical tools to provide insights and alerts about significant market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for AAPL effectively, I need to gather the late

2024-10-28 19:08:44,699 - 13600059392 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 730, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 909, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/util/ssl_.py", line 469, in ssl_wrap_socket
    ssl_soc

In [16]:
from IPython.display import Markdown
Markdown(result)


Risk Analysis Report for AAPL Trading Strategies and Execution Plans:

Introduction:
The proposed trading strategies and execution plans for AAPL involve day trading with a medium risk tolerance. This report aims to provide a comprehensive analysis of potential risks associated with these strategies and recommend mitigation strategies to ensure successful trading outcomes.

Market Analysis:
AAPL shares have shown a 0.86% increase in stock price, reflecting the company's strong performance and market stability. However, factors such as market volatility, regulatory changes impacting tech companies, geopolitical risks, and currency fluctuations pose potential risks to AAPL trading.

Risk Analysis:
1. Market Volatility: The unpredictable nature of stock prices can lead to unexpected losses. Mitigation strategy: Implement stop-loss orders to manage potential losses and monitor market news for any significant changes affecting tech sectors.

2. Regulatory Changes: Changes in regulations can impact the operations and profitability of tech giants like AAPL. Mitigation strategy: Stay informed about regulatory developments and consider hedging strategies using options to protect against adverse regulatory impacts.

3. Geopolitical Risks: AAPL's global presence exposes it to geopolitical risks that can affect stock prices. Mitigation strategy: Diversify the portfolio to reduce exposure to geopolitical risks and consider incorporating geopolitical risk analysis into trading decisions.

4. Currency Fluctuations: Fluctuations in currency exchange rates can impact AAPL's international sales and profitability. Mitigation strategy: Monitor currency movements and consider using currency hedging instruments to mitigate the impact of currency fluctuations on trading outcomes.

Recommendations:
- Set stop-loss orders to manage potential losses and protect capital.
- Stay informed about market news and regulatory developments affecting tech sectors.
- Diversify the portfolio to reduce exposure to geopolitical risks and currency fluctuations.
- Regularly review AAPL's financial health through earnings reports and market analysts' forecasts to adjust trading strategies as needed.

Conclusion:
By implementing the recommended mitigation strategies and staying informed about potential risks, traders can effectively manage risks associated with AAPL trading strategies and execution plans. Success in day trading AAPL with a medium risk tolerance requires a balanced approach that considers market dynamics, regulatory changes, geopolitical risks, and currency fluctuations to achieve profitable trading outcomes.